In [4]:
# Install and import required libraries
!pip install langchain
!pip install huggingface_hub
!pip install sentence_transformers
!pip install -U langchain-community
!pip install pdfminer.six
!pip install pi-heif
!pip install unstructured
!pip install unstructured[local-inference]

In [5]:
from langchain.document_loaders import UnstructuredPDFLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain import HuggingFaceHub
from langchain.chains import RetrievalQA
import os

In [6]:
# Set the Hugging Face API key
os.environ["HUGGINGFACEHUB_API_TOKEN"] = "ENTER YOUR HUGGING FACE API KEY"

In [7]:
# Load a PDF file using UnstructuredPDFLoader
loaders = [UnstructuredPDFLoader("/content/sample.pdf")]
loaders

In [8]:
# Create a vectorstore index from the PDF loaders
index = VectorstoreIndexCreator(
    embedding=HuggingFaceEmbeddings(),
    text_splitter=CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)).from_loaders(loaders)

<ipython-input-8-34cb39369649>:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding=HuggingFaceEmbeddings(),
<ipython-input-8-34cb39369649>:2: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embedding=HuggingFaceEmbeddings(),
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/langchain/indexes/vectorstore.py:128: UserWarning: Using InMemoryVectorStore as the default vectorstore.This memory store won't persist data. You should explicitlyspecify a vectorstore when using VectorstoreIndexCreator
  warnings.warn(


In [9]:
# Load a pre-trained language model from Hugging Face Hub
llm=HuggingFaceHub(repo_id="google/flan-t5-large", model_kwargs={"temperature":0, "max_length":512})

<ipython-input-9-625e874b5430>:1: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEndpoint``.
  llm=HuggingFaceHub(repo_id="google/flan-t5-large", model_kwargs={"temperature":0, "max_length":512})


In [10]:
# Create a RetrievalQA chain that allows for question-answering using a retriever and LLM
chain = RetrievalQA.from_chain_type(llm=llm,
                                    chain_type="stuff",
                                    retriever=index.vectorstore.as_retriever(),
                                    input_key="question")

In [12]:
# Invoke the RetrievalQA chain with a sample question to get an answer
chain.invoke('Whose has prepared this document?')

{'question': 'Whose has prepared this document?',
 'result': 'Accelio Present Applied Technology'}

In [17]:
chain.invoke("How to enable bookmarks in Acrobat?")

{'question': 'How to enable bookmarks in Acrobat?',
 'result': 'Click Window > Show Bookmarks'}

In [18]:
chain.invoke("What sorting criteria are used?")

{'question': 'What sorting criteria are used?',
 'result': 'Invoices sorted, ascending, by date. Invoices sorted, ascending, by item type. Invoices sorted, descending, by transaction amount.'}